# Working with Time Zones

## Why work with Time Zones?

It is fairly common, though erroneous, advice that you should use time zones only for display, and you should immediately convert all datetimes to UTC when working with them.

This is a fine strategy when working with *absolute* times (measuring elapsed time, event ordering, etc), but when working with *wall times* it will cause problems. For example, imagine that we need to represent market close time for a stock market based in New York. Using timezone-aware datetimes, this is trivial:


In [1]:
from datetime import datetime
from dateutil import tz
from dateutil import rrule as rr

WEEKDAYS = (rr.MO, rr.TU, rr.WE, rr.TH, rr.FR)
closing_times = rr.rrule(freq=rr.DAILY, byweekday=WEEKDAYS, byhour=17,
                         dtstart=datetime(2017, 3, 9, 17, tzinfo=tz.gettz("America/New_York")),
                         count=5)

for dt in closing_times:
    print(dt)

2017-03-09 17:00:00-05:00
2017-03-10 17:00:00-05:00
2017-03-13 17:00:00-04:00
2017-03-14 17:00:00-04:00
2017-03-15 17:00:00-04:00


Represented in UTC, however, you can see that it's not a simple rule to express:

In [2]:
for dt in closing_times:
    print(dt.astimezone(tz.UTC))

2017-03-09 22:00:00+00:00
2017-03-10 22:00:00+00:00
2017-03-13 21:00:00+00:00
2017-03-14 21:00:00+00:00
2017-03-15 21:00:00+00:00


Using aware datetimes allows you to outsource the transition logic (and data) to your time zone provider.

## Python's Time Zone Model

The `tzinfo` class is an abstract base class, and you are required to implement three methods:

```python
class tzinfo:
    def tzname(self, dt):
        raise NotImplementedError()

    def utcoffset(self, dt):
        raise NotImplementedError()
        
    def dst(self, dt):
        raise NotImplementedError()
        
    ...
```

- `tzname`: Return the name or abbreviation for the time zone at the given datetime.
- `utcoffset`: Return a `datetime.timedelta` representing the offset from UTC at the given datetime.
- `dst`: Return a `datetime.timedelta` representing the amount of the daylight saving time offset at a given time. *This method is rarely useful*.

The reason these are *methods* rather than fixed values is that a `tzinfo` represents a mapping between a datetime and the set of rules that applies at that time in this zone. The same concrete `tzinfo` class is used for many `datetime`s.

### Example: Mountain Time

In [2]:
from dateutil import tz
from datetime import datetime, timedelta

In [3]:
def print_tzinfo(dt):
    """Convenience function for printing the time zone information of a datetime"""
    print("dt.tzname: " + str(dt.tzname()))
    print("dt.utcoffset: " + str(dt.utcoffset() / timedelta(hours=1)) + " hours")
    print("dt.dst: " + str(dt.dst() / timedelta(hours=1)) + " hours")

In [4]:
MTN = tz.gettz("America/Denver")  # Howdy Wells Fargo Building!

In [5]:
dt_std = datetime(2019, 3, 1, 12, tzinfo=MTN)

In [6]:
print_tzinfo(dt_std)

dt.tzname: MST
dt.utcoffset: -7.0 hours
dt.dst: 0.0 hours


In [7]:
dt_dst = datetime(2019, 4, 1, 12, tzinfo=MTN)

In [8]:
print_tzinfo(dt_dst)

dt.tzname: MDT
dt.utcoffset: -6.0 hours
dt.dst: 1.0 hours


### Example: Implement a UTC time zone class

The simplest `tzinfo` classes to implement are fixed offsets from UTC and UTC itself, which have offsets that do not change as a function of `datetime`. To practice a basic tzinfo implementation, implement a class `UTC` representing the UTC time zone.

In [11]:
import tz_tests

In [12]:
from datetime import tzinfo

In [18]:
class UTC(tzinfo):
    def tz(self, dt):
        return "UTC"
    
    def utcoffset(self, dt):
        return timedelta(0)
    
    def dst(self, dt):
        return None

In [17]:
### Uncomment to test
tz_tests.test_utc(UTC())

NotImplementedError: a tzinfo subclass must implement tzname()